# age|gender预测-word2vec版

In [16]:
import numpy as np
import pandas as pd

In [17]:
import gc

## 载入用户点击日志

In [18]:
pd_user_ad_list = pd.read_csv("./data/pd_user_ad_id_seg_all_click_time.csv")

In [19]:
pd_user_ad_list.head()

,user_id,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,1,20 20 20 39 40 43 46 52 60 64 64 73 76,877468 209778 821396 1683713 122032 71691 1940...,1 1 1 1 1 1 1 1 1 1 1 1 2,773445 188507 724607 1458878 109959 66210 1678...,0 136 0 0 1334 0 0 0 0 64 1454 64 1261,5 2 5 5 2 18 5 5 18 2 2 2 2,29455 9702 7293 14668 11411 14681 17189 367 44...,106 6 326 326 0 326 73 217 64 245 238 245 6
1,2,10 11 14 17 28 28 28 38 38 39 41 42 42 42 44 4...,63441 155822 39714 609050 13069 1266180 441462...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,58788 139702 38066 541125 14495 1107111 392680...,87 80 129 129 1400 0 87 0 1261 0 111 129 0 0 0...,2 2 2 2 2 18 2 18 2 18 2 2 18 18 18 18 2 2 2 2...,22885 10686 18562 25932 768 34505 22885 26006 ...,318 238 6 6 317 47 318 47 6 47 242 6 47 6 47 4...
2,3,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...,661347 808612 593522 825434 710859 726940 3920...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,586668 713448 527601 728308 629278 643108 3502...,36256 40905 1674 35985 1674 0 0 1031 1786 2258...,17 17 2 17 2 18 18 2 2 2 18 12 2 2 18 18 8 4 2...,32974 9877 17018 14186 18492 9058 8371 2336 39...,0 0 322 0 322 6 54 6 322 322 205 302 322 322 2...
3,4,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...,39588 589886 574787 1892854 1230094 31070 2264...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,37966 524312 511235 1638619 1076286 30773 1953...,1862 0 2625 38743 39904 39422 37758 41265 0 15...,2 18 2 17 17 17 4 17 18 2 2 2 2 2 2 2 2 2 5 2 ...,19451 7976 13084 12130 10172 13299 811 23664 1...,238 25 248 0 0 0 0 0 88 319 6 238 319 238 319 ...
4,5,3 13 14 15 20 21 24 25 27 28 29 30 32 32 35 35...,296145 350759 24333 43235 852327 1054434 12964...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,265971 314795 24966 41148 751113 925792 113378...,0 0 87 136 0 136 0 0 0 1064 0 2620 0 0 0 136 2...,5 8 2 2 18 2 5 18 18 2 18 2 5 5 18 2 2 18 18 5...,11882 992 22885 9706 38760 2862 17745 31552 22...,297 0 318 6 322 6 288 322 319 238 322 322 203 ...


In [20]:
pd_user_ad_list.shape

(1900000, 9)

### 标签信息

In [21]:
df_user_label = pd.read_csv("./data/train_preliminary/user.csv")

In [22]:
df_user_label.shape

(900000, 3)

#### 分离出测试集用户id

In [23]:
user_ids_test = set(pd_user_ad_list['user_id'].unique())-set(df_user_label['user_id'].values)

In [24]:
len(user_ids_test)

1000000

In [25]:
df_test_user_id = pd.DataFrame(data=user_ids_test,columns=['user_id'])

In [26]:
df_test_user_id.shape

(1000000, 1)

In [27]:
df_test_user_id['user_id'].min(),df_test_user_id['user_id'].max()

(3000001, 4000000)

In [28]:
df_user_label['user_id'].min(),df_user_label['user_id'].max()

(1, 900000)

## 训练词向量

### w2v

In [30]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import word2vec

In [ ]:
#pd_user_ad_list[['id_segcontent']].to_csv("./data/creative_id_seg_content.csv",header=False,index=False)

In [40]:
def get_seg_list(dfData,col_name):
    '''
    获取指定id类型的用户id序列
    Parameters
    ----------
    dfData：pandas.DataFrame，需包含列名['user_id','time','creative_id','click_times','ad_id','product_id','product_category','advertiser_id','industry']
    col_name：str,列名
    Returns
    ----------
    cr_segcontent_list：list,用户id序列
    '''
    cr_segcontent_list=[]
    count=0
    for segcontent in dfData[col_name].values:
        count+=1
        cr_segcontent_list.append(segcontent.split(" "))
        if count%100000==0:
            print(count)
    return cr_segcontent_list

#### 素材id

In [34]:
cr_segcontent_list=get_seg_list(pd_user_ad_list,'creative_id')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [35]:
cr_segcontent_list[0]

['877468',
 '209778',
 '821396',
 '1683713',
 '122032',
 '71691',
 '1940159',
 '90171',
 '2087846',
 '2456345',
 '3068256',
 '2456345',
 '3592928']

训练词向量

In [123]:
wv_model_cr = Word2Vec(cr_segcontent_list,size=128,sg=1, window=100, min_count=1, iter=10,workers=24)

In [25]:
wv_model_cr.wv.vector_size

128

保存词向量

In [124]:
wv_model_cr.wv.save("./model/word2vec/kv_cr/tx_gender_age_log_sg_win100_neg5_dim128_iter10_minc1_w2v_cr.kv")

载入词向量

In [38]:
wv_cr = KeyedVectors.load("./model/word2vec/kv_cr/tx_gender_age_log_sg_win100_neg5_dim128_iter10_minc1_w2v_cr.kv", mmap='r')

In [118]:
#del wv_cr

In [126]:
del wv_model_cr

In [127]:
gc.collect()

0

In [44]:
type(wv_cr['877468'])

numpy.memmap

In [45]:
len(wv_cr.vocab.keys())

3412772

#### 广告id

In [59]:
ad_segcontent_list = get_seg_list(pd_user_ad_list,'ad_id')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [7]:
ad_segcontent_list[0]

['773445',
 '188507',
 '724607',
 '1458878',
 '109959',
 '66210',
 '1678901',
 '82421',
 '1803758',
 '2116146',
 '2638858',
 '2116146',
 '3093461']

In [8]:
wv_model_ad = Word2Vec(ad_segcontent_list,size=128,sg=1, window=10, min_count=1, iter=10,workers=24)

In [123]:
wv_model_ad.wv.vector_size

128

In [124]:
len(wv_model_ad.wv.vocab.keys())

3027360

In [49]:
#另一种保存方式，保存整个模型
#wv_model_ad.save("./model/word2vec/tx_gender_age_log_sg_win10_neg5_dim128_iter10_minc1_w2v_ad.model")

In [10]:
wv_model_ad.wv.save("./model/word2vec/kv_ad/tx_gender_age_log_sg_win10_neg5_dim128_iter10_minc1_w2v_ad.kv")

In [46]:
wv_ad =  KeyedVectors.load("./model/word2vec/kv_ad/tx_gender_age_log_sg_win10_neg5_dim128_iter10_minc1_w2v_ad.kv", mmap='r')

In [12]:
del wv_model_ad

In [13]:
gc.collect()

0

In [47]:
wv_ad.vector_size

128

In [48]:
len(wv_ad.vocab.keys())

3027360

In [30]:
wv_ad['773445']

memmap([ 0.52969337,  0.33892685, -0.0703271 , -0.34758982, -0.35821044,
        -0.21997395,  0.49055594,  0.42249727,  0.24383329, -0.45413095,
        -0.75186896,  0.45233342,  0.03626498,  0.2921691 , -0.34779158,
        -0.68172795, -0.628323  , -0.73124325, -0.06407569,  0.13663574,
        -0.11289886, -0.5851949 , -0.5324513 , -0.3608619 , -0.03872818,
        -0.07542732,  0.3897827 ,  0.35616758,  0.33299232, -0.00696155,
         0.04148217, -0.15156786, -0.07657099,  0.31003407, -0.35803583,
         0.36154985,  0.03016107,  0.22941351,  0.05970838,  0.21700382,
         0.27672017, -0.05959861,  0.32186624, -0.07051835, -0.04180244,
         0.05897556, -0.31905413,  0.10865369, -0.44322863,  0.15936883,
         0.12831885, -0.26384902, -0.6091436 , -1.0872767 ,  0.3676072 ,
         0.14638689,  0.20025156, -0.20940013,  0.5903393 , -0.04398829,
         0.20712706, -0.19877014, -0.0698178 , -0.03146586,  0.35813892,
        -0.26679087, -0.16335687,  0.59887457,  0.6

#### 广告主id

In [60]:
adv_segcontent_list=get_seg_list(pd_user_ad_list,'advertiser_id')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [14]:
adv_segcontent_list[0]

['29455',
 '9702',
 '7293',
 '14668',
 '11411',
 '14681',
 '17189',
 '367',
 '44865',
 '188',
 '23575',
 '188',
 '10988']

In [108]:
wv_model_adv = Word2Vec(adv_segcontent_list,sg=1,size=64, window=10, min_count=1, iter=10,workers=24)

In [36]:
'29455' in wv_model_adv.wv

True

In [19]:
#wv_model_adv.save("./model/word2vec/tx_gender_age_log_sg_win10_neg5_dim64_iter10_minc1_w2v_adv.model")

In [ ]:
wv_model_adv.wv.save("./model/word2vec/kv_adv/tx_gender_age_log_sg_win10_neg5_dim64_iter10_minc1_w2v_adv.kv")

In [50]:
wv_adv = KeyedVectors.load("./model/word2vec/kv_adv/tx_gender_age_log_sg_win10_neg5_dim64_iter10_minc1_w2v_adv.kv", mmap='r')

In [ ]:
del wv_model_adv

In [ ]:
gc.collect()

#### 产品id

In [61]:
pr_segcontent_list=get_seg_list(pd_user_ad_list,'product_id')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [93]:
wv_model_pr = Word2Vec(pr_segcontent_list,sg=1,size=64, window=10, min_count=1, iter=10,workers=24)

In [22]:
#wv_model_pr.save("./model/word2vec/tx_gender_age_log_sg_win10_neg5_dim64_iter10_minc1_w2v_pr.model")

In [94]:
wv_model_pr.wv.save("./model/word2vec/kv_pr/tx_gender_age_log_sg_win10_neg5_dim64_iter10_minc1_w2v_pr.kv")

In [51]:
wv_pr = KeyedVectors.load("./model/word2vec/kv_pr/tx_gender_age_log_sg_win10_neg5_dim64_iter10_minc1_w2v_pr.kv", mmap='r')

In [ ]:
del wv_model_pr

In [36]:
gc.collect()

715

**词向量预训练至此结束**

## 获取用户向量

### 方式一

In [39]:
gc.collect()

16

In [41]:
def user_w2v(df_data,wv,col_name):
    """
    生成用户的词向量表征（考虑点击次数）
    
    Parameters
    ----------
    df_data: pandas.DataFrame,数据源
    wv: gensim.models.keyedvectors.Word2VecKeyedVectors,训练好的词向量
    col_name: str,列名
    
    Returns
    ----------
    numpy.ndarray, 用户词向量表征
    """
    k = wv.vector_size
    user_vectors=[]
    count=0
    for index,row in df_data.iterrows():
        
        count+=1
        user_vec = np.zeros(k)
        
        id_segcontent = row[col_name]
        click_times = row['click_times']
                          
        ad_list = id_segcontent.split(" ")
        click_time_list = click_times.split(" ")

        word_len = len(ad_list)
        word_size = 0
        for i in range(0,word_len):
            if ad_list[i] in wv:
                user_vec+=wv[ad_list[i]]*int(click_time_list[i])
                word_size+=int(click_time_list[i])
        user_vec = user_vec/word_size
        user_vectors.append(user_vec)
        if count%100000==0:
            print(count)
    return np.array(user_vectors)

In [13]:
user_vectors_cr = user_w2v(pd_user_ad_list,wv_cr,col_name='creative_id')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [10]:
pd_user_ad_list.shape[0]

1900000

In [7]:
pd_user_ad_list['creative_id'][0]

'877468 209778 821396 1683713 122032 71691 1940159 90171 2087846 2456345 3068256 2456345 3592928'

### 方式二

In [55]:
import collections

In [56]:
def user_w2v(segcontent_list,wv):
    """
    生成用户的词向量表征（不考虑点击次数）
    
    Parameters
    ----------
    segcontent_list: 用户id序列
    wv: gensim.models.keyedvectors.Word2VecKeyedVectors,训练好的词向量
    
    Returns
    ----------
    numpy.ndarray, 用户词向量表征
    """
    k = wv.vector_size
    user_vectors=[]
    count=0
    for user_segcontent in segcontent_list:
        count+=1
        user_vec = np.zeros(k)
        ad_count_dict = collections.Counter(user_segcontent)
        word_size=0
        for ad_id in ad_count_dict.keys():
            if ad_id in wv:
                user_vec+=wv[ad_id]*ad_count_dict[ad_id]
                word_size+=ad_count_dict[ad_id]
        user_vec = user_vec/word_size
        user_vectors.append(user_vec)
        if count%100000==0:
            print(count)
    return np.array(user_vectors)

In [57]:
user_vectors_cr = user_w2v(cr_segcontent_list,wv=wv_cr)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [62]:
user_vectors_ad = user_w2v(ad_segcontent_list,wv=wv_ad)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [63]:
user_vectors_adv = user_w2v(adv_segcontent_list,wv=wv_adv)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [64]:
user_vectors_pr = user_w2v(pr_segcontent_list,wv=wv_pr)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [67]:
user_vectors_cr.shape,user_vectors_ad.shape,user_vectors_adv.shape,user_vectors_pr.shape

((1900000, 128), (1900000, 128), (1900000, 64), (1900000, 64))

### doc2vec

此部分在目前场景下效果不佳，仅供参考

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import doc2vec

In [ ]:
from gensim.test.utils import datapath

In [ ]:
from gensim.test.utils import common_texts

In [ ]:
common_texts

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(segcontent_list)]

In [ ]:
%time dv_model_dbow = Doc2Vec(documents,dm=0,vector_size=128,window=5, min_count=1,hs=0,negative=5,sample=0.001,workers=32,epochs=10)

In [ ]:
del dv_model_dbow

In [305]:
dv_model_dbow.save("./model/tx_gender_age_log_win5_mc1_neg5_dim128_iter10_dbow_d2v.model")

In [291]:
dv_model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [ ]:
len(documents)

In [289]:
dv_model_dbow.docvecs[0]

array([-0.00881683, -0.09074552,  0.13075437, -0.12135855,  0.09921252,
       -0.01294636, -0.08020144, -0.05435244,  0.01543843,  0.35401312,
        0.1546657 , -0.21313967, -0.10770545, -0.21218939,  0.03634015,
       -0.08141699, -0.00262357,  0.05173508,  0.04500632, -0.05160706,
        0.12433016,  0.03802612,  0.06809052, -0.0615447 ,  0.10139114,
        0.15431184, -0.01228508,  0.12595516, -0.18911907, -0.14850868,
        0.0488173 ,  0.02910751,  0.1443899 , -0.22144215, -0.07177097,
        0.2585742 , -0.03468794, -0.01290978,  0.16182937, -0.14685196,
       -0.00163103,  0.22421774, -0.05435663,  0.05489495,  0.19467843,
        0.07843263, -0.06721466, -0.26648575, -0.03835699,  0.20915097,
       -0.07001007,  0.12921974,  0.02046305,  0.07181315, -0.00357183,
        0.06828393, -0.11953515,  0.08324942, -0.04401371,  0.10308376,
        0.1904173 , -0.06379346, -0.08766186,  0.07839777, -0.0344345 ,
        0.06396211, -0.06207786, -0.09146874,  0.06072115,  0.02

In [ ]:
dv_model_dbow.wv.vector_size

In [296]:
def user_doc2vec(l,dv_model):
    k = dv_model.wv.vector_size
    user_vectors=[]
    count=0
    for i in range(0,l):
        count+=1
        #使用user_vectors.append(dv_model.infer_vector(ad_list))会让模型继续训练，因而很慢
        user_vectors.append(dv_model.docvecs[i])
        if count%100000==0:
            print(count)
    return np.array(user_vectors)

In [297]:
user_vectors = user_doc2vec(len(documents),dv_model=dv_model_dbow)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


## 构造样本

In [22]:
#pd_word2_vec = pd_word2_vec_ad

In [66]:
w2v_fname=['w2v_cr_'+str(x) for x in range(0,wv_cr.vector_size)]

In [68]:
pd_word2vec_cr = pd.DataFrame(user_vectors_cr,columns=w2v_fname)
pd_word2vec_cr = pd.concat([pd_user_ad_list[["user_id"]],pd_word2vec_cr],axis=1)

In [69]:
w2v_fname=['w2v_ad_'+str(x) for x in range(0,wv_ad.vector_size)]

In [70]:
pd_word2vec_ad = pd.DataFrame(user_vectors_ad,columns=w2v_fname)
pd_word2vec_ad = pd.concat([pd_user_ad_list[["user_id"]],pd_word2vec_ad],axis=1)

In [71]:
w2v_fname=['w2v_adv_'+str(x) for x in range(0,wv_adv.vector_size)]

In [72]:
pd_word2vec_adv = pd.DataFrame(user_vectors_adv,columns=w2v_fname)
pd_word2vec_adv = pd.concat([pd_user_ad_list[["user_id"]],pd_word2vec_adv],axis=1)

In [73]:
w2v_fname=['w2v_pr_'+str(x) for x in range(0,wv_pr.vector_size)]

In [74]:
pd_word2vec_pr = pd.DataFrame(user_vectors_pr,columns=w2v_fname)
pd_word2vec_pr = pd.concat([pd_user_ad_list[["user_id"]],pd_word2vec_pr],axis=1)

In [75]:
pd_word2vec_cr.shape,pd_word2vec_ad.shape,pd_word2vec_adv.shape,pd_word2vec_pr.shape

((1900000, 129), (1900000, 129), (1900000, 65), (1900000, 65))

导出部分词向量特征

In [ ]:
pd_word2vec_cr.to_csv("./data/user_word2_vec_128.csv",index=False)

In [49]:
pd_word2vec_cr = pd.read_csv("./data/user_word2_vec_128.csv")

In [50]:
pd_word2vec_cr.head()

,user_id,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,...,w2v_118,w2v_119,w2v_120,w2v_121,w2v_122,w2v_123,w2v_124,w2v_125,w2v_126,w2v_127
0,1,-0.193146,-0.228589,-0.212177,0.502402,0.419108,-0.155171,-0.050705,-0.278514,-0.257985,...,-0.508188,-0.047964,0.173882,-0.233017,-0.106127,0.137993,0.364915,-0.258417,0.158040,-0.312584
1,2,-0.427113,-0.449429,0.155294,0.204986,0.057536,0.182058,-0.118976,-0.134916,0.312591,...,-0.383222,0.240402,0.421468,-0.091944,0.024992,0.344723,0.350942,-0.021163,-0.156320,0.067091
2,3,-0.290485,-0.123267,0.147264,0.294396,0.333517,-0.002210,-0.147300,-0.487144,0.015274,...,-0.172198,0.003900,0.099584,-0.028236,-0.022920,0.151936,0.563423,0.020704,-0.102448,0.014376
3,4,-0.258450,-0.598861,0.174674,0.272897,0.258750,0.102361,-0.110127,-0.355300,0.377514,...,-0.160130,0.139451,-0.130731,-0.239497,0.049233,0.100909,0.297746,0.005558,-0.144150,-0.023132
4,5,-0.403251,-0.223418,-0.140462,0.211255,0.477512,0.247194,-0.021708,-0.284819,-0.154669,...,-0.323597,-0.099868,0.383555,-0.227641,-0.094448,0.155718,0.490123,-0.108260,-0.032434,-0.036300


### 融合词向量特征

In [76]:
pd_word2vec_cr.shape,pd_word2vec_ad.shape,pd_word2vec_adv.shape,pd_word2vec_pr.shape

((1900000, 129), (1900000, 129), (1900000, 65), (1900000, 65))

In [77]:
pd_word2vec = pd.merge(pd_word2vec_cr,pd_word2vec_ad,on='user_id')

In [78]:
pd_word2vec = pd.merge(pd_word2vec,pd_word2vec_adv,on='user_id')

In [79]:
pd_word2vec = pd.merge(pd_word2vec,pd_word2vec_pr,on='user_id')

In [80]:
pd_word2vec.shape

(1900000, 385)

In [81]:
gc.collect()

0

In [82]:
#del df_sample_train

In [83]:
df_sample_train = pd.merge(pd_word2vec,df_user_label,on='user_id')

In [84]:
df_sample_train.shape

(900000, 387)

In [85]:
df_sample_train.head()

,user_id,w2v_cr_0,w2v_cr_1,w2v_cr_2,w2v_cr_3,w2v_cr_4,w2v_cr_5,w2v_cr_6,w2v_cr_7,w2v_cr_8,...,w2v_pr_56,w2v_pr_57,w2v_pr_58,w2v_pr_59,w2v_pr_60,w2v_pr_61,w2v_pr_62,w2v_pr_63,age,gender
0,1,0.089777,-0.132495,0.222717,0.312500,-0.089935,-0.035498,0.219225,0.380090,-0.098880,...,-0.102909,-0.069506,-0.013462,-0.303535,-0.151102,-0.416932,-0.274069,0.271734,4,1
1,2,0.205643,-0.154495,0.237451,0.158883,-0.008960,-0.104230,0.023858,0.343034,0.014614,...,-0.011438,-0.113968,-0.066190,-0.297549,-0.079522,-0.305683,-0.199420,0.353566,10,1
2,3,0.019845,-0.397888,0.222115,0.410672,-0.006792,-0.000501,0.163669,0.344088,-0.078659,...,-0.211163,-0.059042,-0.065321,-0.379983,-0.123349,-0.292780,-0.320767,0.087188,7,2
3,4,0.046794,-0.229000,0.067148,0.382606,0.176876,-0.121723,0.071203,0.216840,0.084766,...,-0.183278,-0.143893,-0.063736,-0.417863,-0.030431,-0.182565,-0.275010,0.154389,5,1
4,5,0.114058,-0.010847,0.189893,-0.046693,-0.158460,0.046259,0.472983,0.323280,-0.029474,...,0.056306,-0.133631,-0.105429,-0.292214,-0.190382,-0.420457,-0.333188,0.229452,4,1


In [86]:
df_sample_test = pd.merge(pd_word2vec,df_test_user_id,on='user_id')

In [87]:
df_sample_test.shape

(1000000, 385)

In [ ]:
feature_exclude_columns=['user_id','age','gender']

In [ ]:
feature_columns = [x for x in df_sample_train.columns if x not in feature_exclude_columns]

In [ ]:
len(feature_columns)

In [ ]:
gc.collect()

## 建模测试

In [33]:
from sklearn.model_selection import train_test_split
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import accuracy_score

In [34]:
from sklearn.model_selection import cross_val_score

In [35]:
from sklearn.externals import joblib

/opt/app/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### 性别

In [36]:
lgbm_gender = LGBMClassifier(n_estimators=300,learning_rate=0.1,max_depth=8,num_leaves=127,subsample=0.8,colsample_bytree=0.8,min_split_gain=3,random_state=0,reg_alpha=0.05)

In [37]:
def model_test(clf,df_data,feature_columns,label_name,test_size=0.3,refit=False):
     '''
    建模测试
    Parameters
    ----------
    clf:sklearn model
    df_data：pandas.DataFrame,训练数据源
    feature_columns：list,特征列名
    label_name：str,标签列名
    test_size：验证集比例
    refit：boolean,测试完是否refit
    Returns
    ----------
    clf:sklearn model
    '''
    X_train, X_test, y_train, y_test = train_test_split(df_data[feature_columns],df_data[label_name],test_size=test_size,random_state=0)
    print("数据集划分完毕")
    clf.fit(X_train,y_train)
    print("模型训练完毕")
    y_test_predict = clf.predict(X_test)
    y_train_predict = clf.predict(X_train)
    print("训练集：%f,测试集：%f" % (accuracy_score(y_train,y_train_predict),accuracy_score(y_test,y_test_predict)))
    if refit is True:
        clf.fit(df_data[feature_columns],df_data[label_name])
    return clf

In [63]:
X_train, X_test, y_train, y_test = train_test_split(df_sample_train[feature_columns],df_sample_train['gender'],test_size=0.3,random_state=0)

单id

cr:size=128,window=5

In [61]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.945535,测试集：0.920344


cr:size=128,window=10

In [112]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.946943,测试集：0.922948


cr:size=256,window=10

In [140]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.949924,测试集：0.922041


cr:size=128,window=100

In [34]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.947544,测试集：0.924763


adv:size=128,window=10

In [92]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.918963,测试集：0.890222


adv:size=64:window=10

In [51]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.913338,测试集：0.889204


adv:size=64,window=100

In [122]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.911878,测试集：0.888263


adv:size=32,window=10

In [62]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.902352,测试集：0.884437


pr:size=64,window=10

In [107]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.874917,测试集：0.850715


pr:size=16,window=10

In [77]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.855970,测试集：0.845863


ad:size=128,window=10

In [140]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.947221,测试集：0.922900


ad:size=128,window=100

In [38]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.947695,测试集：0.924970


多id

cr+pr+adv

In [92]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.948065,测试集：0.922163


cr+ad+pr+adv

In [164]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.951481,测试集：0.925904


cr+ad

In [171]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.949968,测试集：0.924556


refit

In [ ]:
lgbm_gender.fit(df_sample_train[feature_columns],df_sample_train['gender'])

In [ ]:
y_test_predict = lgbm_gender.predict(X_test)
y_train_predict = lgbm_gender.predict(X_train)

In [ ]:
accuracy_score(y_train,y_train_predict),accuracy_score(y_test,y_test_predict)

In [66]:
gc.collect()

16

### 年龄

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_sample_train[feature_columns],df_sample_train['age'],test_size=0.3,random_state=0)

In [39]:
lgbm_age =LGBMClassifier(n_estimators=300,learning_rate=0.1,max_depth=8,num_leaves=127,subsample=0.8,colsample_bytree=0.8,min_split_gain=3,random_state=0,reg_alpha=0.05)

In [99]:
lgbm_age = model_test(lgbm_age,df_sample_train,feature_columns=feature_columns,label_name='age')

数据集划分完毕
模型训练完毕
训练集：0.554054,测试集：0.378707


refit

In [ ]:
lgbm_age.fit(df_sample_train[feature_columns],df_sample_train['age'])

In [ ]:
y_test_predict = lgbm_age.predict(X_test)
y_train_predict = lgbm_age.predict(X_train)

In [ ]:
accuracy_score(y_train,y_train_predict),accuracy_score(y_test,y_test_predict)

## 模型固化

In [ ]:
joblib.dump(lgbm_gender,'./model/lgbm_gender_w2v128.pkl')

In [62]:
lgbm_gender = joblib.load('./model/lgbm_gender_w2v128.pkl')

In [64]:
y_test_predict = lgbm_gender.predict(X_test)
y_train_predict = lgbm_gender.predict(X_train)

In [65]:
accuracy_score(y_train,y_train_predict),accuracy_score(y_test,y_test_predict)

(0.9401746031746032, 0.9394037037037037)

In [ ]:
joblib.dump(lgbm_age,'./model/lgbm_age_w2v128.pkl')

## 测试集输出结果

In [ ]:
train_result_gender = lgbm_gender.predict(df_sample_train[feature_columns])

In [ ]:
accuracy_score(df_sample_train['gender'],train_result_gender)

In [ ]:
test_result_gender = lgbm_gender.predict(df_sample_test[feature_columns])

In [ ]:
df_sample_test['predicted_gender'] = test_result_gender

In [ ]:
train_result_age = lgbm_age.predict(df_sample_train[feature_columns])

In [ ]:
accuracy_score(df_sample_train['age'],train_result_age)

In [ ]:
test_result_age = lgbm_age.predict(df_sample_test[feature_columns])

In [ ]:
df_sample_test['predicted_age'] = test_result_age

In [ ]:
df_sample_test[['user_id','predicted_age','predicted_gender']].to_csv("./result/submission.csv",index=False)